In [2]:
#importing openslide:
OPENSLIDE_PATH = r'C:\Users\Kevin\Documents\openslide-win64-20221217\bin'

import os

if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [4]:
import os
vipshome = r'C:\Users\Kevin\Documents\vips-dev-8.14\bin'
os.environ['PATH'] = vipshome + ';' + os.environ['PATH']
import pyvips

In [1]:
import numpy as np
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import cv2
import os
from matplotlib import pyplot as plt
from skimage.morphology import remove_small_objects, remove_small_holes
from skimage.color import rgb2gray
from skimage.transform import warp
from skimage.registration import optical_flow_tvl1, optical_flow_ilk
import xml.etree.ElementTree as ET
import pandas as pd
import skimage.measure
import scipy.stats as stats
import time
import shutil
from tqdm import tqdm
from skimage.measure import label, regionprops_table

In [3]:
# src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\Unstained'
# imnm = 'OTS_14684_3.ndpi'
# imobj = openslide.open_slide(os.path.join(src,imnm))
# imobj = imobj.read_region(location=(0,0),level=1,size=imobj.level_dimensions[1])
# imobj.level_dimensions[1]

In [3]:
#wsi to tiles
def reshape_split(image:np.ndarray,kernel_size:tuple):
    img_height,img_width,channels=image.shape
    tile_height,tile_width = kernel_size
    tiled_array = image.reshape(img_height//tile_height,
                                tile_height,
                                img_width//tile_width,
                                tile_width,
                                channels)
    tiled_array = tiled_array.swapaxes(1,2)
    return tiled_array

In [ ]:
src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\Unstained'
imnm = 'OTS_14832_3.ndpi'
imobj = openslide.open_slide(os.path.join(src,imnm))
imobj = imobj.read_region(location=(0,0),level=1,size=imobj.level_dimensions[1])

# Image to Array
imnp = np.array(imobj)
imobj.close()
h,w,_=imnp.shape
tile_height, tile_width = (1024,1024)
# Padding (so that it's in multiples of 1024)
imnpr = np.pad(imnp, pad_width=[(0, tile_height-h%tile_height),(0, tile_width-w%tile_width),(0, 0)], mode='constant', constant_values=0)
# imnpr = imnpr / 127.5 - 1 #normalize [-1 1]
# imnpr = imnpr / 255 #normalize [0 1]
img_height2,img_width2,channels=imnpr.shape
# Tile
tiles = reshape_split(imnpr, (1024,1024))

In [87]:
print(tiles.shape)
nrows = tiles.shape[0]
ncols = tiles.shape[1]

(54, 111, 1024, 1024, 4)


In [88]:
imnp.shape

(54912, 113280, 4)

In [89]:
imnpr.shape

(55296, 113664, 4)

In [15]:
# save_src = r'\\shelter\Kyu\unstain2stain\tiles\not_registrated\Unstained'
# for idx in range(0,tiles.shape[0]):
#     for idx1 in range(0,tiles.shape[1]):
#         tmp_tile = tiles[idx][idx1]
#         save_path = os.path.join(save_src,str(idx)+str(idx1)+'tile'+'.png')
#         Image.fromarray(tmp_tile).save(save_path)

In [16]:
src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE'
imnm = 'OTS_14832_3_he.ndpi'
imobj = openslide.open_slide(os.path.join(src,imnm))
imobj = imobj.read_region(location=(0,0),level=1,size=imobj.level_dimensions[1])
# Image to Array
imnp = np.array(imobj)
imobj.close()
h,w,_=imnp.shape
tile_height, tile_width = (1024,1024)
# Padding
imnpr = np.pad(imnp, pad_width=[(0, tile_height-h%tile_height),(0, tile_width-w%tile_width),(0, 0)], mode='constant', constant_values=0)
# imnpr = imnpr / 127.5 - 1 #normalize [-1 1]
# imnpr = imnpr / 255 #normalize [0 1]
img_height2,img_width2,channels=imnpr.shape
# Tile
tiles = reshape_split(imnpr, (1024,1024))

In [18]:
# save_src = r'\\shelter\Kyu\unstain2stain\tiles\not_registrated\HE'
# for idx in range(0,tiles.shape[0]):
#     for idx1 in range(0,tiles.shape[1]):
#         tmp_tile = tiles[idx][idx1]
#         save_path = os.path.join(save_src,str(idx)+str(idx1)+'tile'+'.png')
#         Image.fromarray(tmp_tile).save(save_path)

### The tile generation code above is unregistered, I ran a code in matlab to generate the registered tiles, which have almost the same # of tiles, except that there are some missing images. Find the difference:

In [90]:
stain_tile_src = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\HE\OTS_14832_3_he'
unstain_tile_src = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\Unstained\OTS_14832_3'

In [91]:
unstain_tile_name = os.listdir(unstain_tile_src)
stain_tile_name = os.listdir(stain_tile_src)

In [92]:
print(len(stain_tile_name))
print(len(unstain_tile_name))

10996
10996


In [93]:
unstain_tile_base = [os.path.basename(x) for x in unstain_tile_name]
stain_tile_base = [os.path.basename(x) for x in stain_tile_name]
unstain_list = [ x[-10:-4] for x in unstain_tile_base]
stain_list = [ x[-10:-4] for x in stain_tile_base]

In [94]:
not_in_stain = [x for x in unstain_list if x not in stain_list]
not_in_stain

[]

### Found the three differences- delete them from unstained, and run inference on pix2pix project.

# After inference on pix2pix, first select only wanted tiles (only fake) and save these inferred tiles to the server:

In [3]:
inferred_src = r'C:\Users\Kevin\PycharmProjects\pix2pix\pytorch-CycleGAN-and-pix2pix\results\unstain2stain_cyclegan\test_pix2pix_latest\images' # always output inference is this filepath
inferred_list = os.listdir(inferred_src)
inferred_name = [os.path.basename(x) for x in inferred_list]
real_inferred2 = [x for x in inferred_name if x[-8:-4]  == 'fake']
real_inferred_src = [os.path.join(inferred_src,x) for x in real_inferred2]
print(len(real_inferred_src)) #check expected value

9890


In [4]:
real_inferred_src[0]

'C:\\Users\\Kevin\\PycharmProjects\\pix2pix\\pytorch-CycleGAN-and-pix2pix\\results\\unstain2stain_cyclegan\\test_pix2pix_latest\\images\\100879_10382xy5425_fake.png'

In [5]:
# move the list of inferred_src to server:
destination = r'\\shelter\Kyu\unstain2stain\generated_fake_images\inferred_tiles\fake_OTS_14832_4he'

for idx in tqdm(real_inferred_src,desc="Number of images processed", colour = 'red'):
    shutil.copy(idx, destination)

Number of images processed: 100%|██████████| 9890/9890 [26:36<00:00,  6.20it/s]


In [6]:
len(os.listdir(destination))

9890

# ENTIRE WORKFLOW TO STITCH TILES TO WHOLE SLIDE IMAGE:

In [1]:
import os

vipshome = r'C:\Users\Kevin\Downloads\vips-dev-8.14\bin' #download libvips for windows, this is the bin path
os.environ['PATH'] = vipshome + ';' + os.environ['PATH']
import pyvips
import numpy as np
import os
from PIL import Image

Image.MAX_IMAGE_PIXELS = None
import cv2
import os
from skimage.transform import warp
from skimage.registration import optical_flow_tvl1, optical_flow_ilk
import xml.etree.ElementTree as ET
import pandas as pd
import time
import shutil
from tqdm import tqdm

In [2]:
def return_row_col(image_dir):
    imlist = [x for x in os.listdir(image_dir) if x.endswith('.png')]
    min_x = float('inf')
    min_y = float('inf')
    max_x = float('-inf')
    max_y = float('-inf')
    start = time.time()
    for filename in tqdm(imlist, desc="Number of images processed", colour = 'red'):
        x, y = map(int, filename.split('xy')[0].split('.png')[0].split('_'))
        if x < min_x:
            min_x = x
        if y < min_y:
            min_y = y
        if x > max_x:
            max_x = x
        if y > max_y:
            max_y = y
    end = time.time()
    max_min_ra = min_x, min_y, max_x, max_y
    num_row = int((max_x + 1024 - min_x)/1024)
    num_col = int((max_y + 1024 - min_y)/1024)
    print("time it took: {} minutes".format((round(end-start)/60),3))
    return max_min_ra, num_row, num_col

In [8]:
# destination = r'\\shelter\Kyu\unstain2stain\generated_fake_images\inferred_tiles\fake_OTS_14832_4he'
# tile_src = destination
# max_min_ra, num_row, num_col = return_row_col(tile_src)
# min_x = max_min_ra[0]
# min_y = max_min_ra[1]
# print("min_x is {}, min_y is {}, num_row is {}, num_col is {}".format(min_x, min_y, num_row, num_col))

Number of images processed: 100%|██████████| 9890/9890 [00:00<00:00, 1648321.81it/s]

time it took: 0.0 minutes
min_x is 36367, min_y is 4238, num_row is 115, num_col is 86


In [3]:
def generate_5d_array(image_dir, min_x, min_y, num_rows, num_cols):
    array_5d = np.zeros((num_cols,num_rows, 1024, 1024, 3), dtype=np.uint8)
    imlist = [x for x in os.listdir(image_dir) if x.endswith('.png')]
    start = time.time()
    for filename in tqdm(imlist, desc="Number of images processed", colour = 'red'):
        x, y = map(int, filename.split('xy')[0].split('.png')[0].split('_'))
        x = int((x-min_x)/1024)
        y = int((y-min_y)/1024)
        image = Image.open(os.path.join(image_dir, filename))
        image = np.array(image)
        array_5d[y,x, :, :, :] = image
    end = time.time()
    print("time it took: {} minutes".format((round(end-start)/60),3))
    return array_5d

In [10]:
# tiles = generate_5d_array(image_dir = tile_src,min_x = min_x,min_y = min_y,num_rows = num_row,num_cols = num_col)
# print("shape of tiles is {}".format(tiles.shape))

Number of images processed: 100%|██████████| 9890/9890 [34:40<00:00,  4.75it/s]  

time it took: 34.666666666666664 minutes
shape of tiles is (86, 115, 1024, 1024, 3)


In [11]:
# stitched_wsi = np.reshape(tiles.swapaxes(1,2),(1024*num_col,1024*num_row,3))
# print("shape of reconstructed wsi is {}".format(stitched_wsi.shape))

shape of reconstructed wsi is (88064, 117760, 3)


In [12]:
# start = time.time()
# im = pyvips.Image.new_from_array(stitched_wsi)
# image_height = im.height
# image_bands = im.bands
# print("imheight imwidth",image_height, im.width)
# im = im.copy()
# im.set_type(pyvips.GValue.gint_type, "page-height", image_height)
# im.set_type(pyvips.GValue.gstr_type, "image-description",
#             f"""<?xml version="1.0" encoding="UTF-8"?>
# <OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
#     xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
#     xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
#     <Image ID="Image:0">
#         <!-- Minimum required fields about image dimensions -->
#         <Pixels DimensionOrder="XYCZT"
#                 ID="Pixels:0"
#                 SizeC="{image_bands}"
#                 SizeT="1"
#                 SizeX="{im.width}"
#                 SizeY="{image_height}"
#                 SizeZ="1"
#                 Type="uint8">
#         </Pixels>
#     </Image>
# </OME>""")
# src = r'\\shelter\Kyu\unstain2stain\generated_fake_images\fake_images'
# end = time.time()
# print("time it took to set ometiff up took {} minutes".format((round(end-start)/60),3))
#
# start = time.time()
# im.tiffsave(os.path.join(src, 'cyclegan_OTS_14832_4_fake_HE.ome.tiff'), compression="jpeg", tile=True, tile_width=512,
#             tile_height=512, pyramid=True, subifd=True)
# end = time.time()
# print("time it took to save ometiff took {} minutes".format((round(end-start)/60),3))

imheight imwidth 88064 117760
time it took to set ometiff up took 0.0 minutes
time it took to save ometiff took 7.55 minutes


In [4]:
### Applying above stitch workflow to create a reconstructed ometiff from registered tiles (not inferred tiles):
tile_src = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\HE\OTS_14832_10_he'
max_min_ra, num_row, num_col = return_row_col(tile_src)
min_x = max_min_ra[0]
min_y = max_min_ra[1]
print("min_x is {}, min_y is {}, num_row is {}, num_col is {}".format(min_x, min_y, num_row, num_col))

tiles = generate_5d_array(image_dir = tile_src,min_x = min_x,min_y = min_y,num_rows = num_row,num_cols = num_col)
print("shape of tiles is {}".format(tiles.shape))

stitched_wsi = np.reshape(tiles.swapaxes(1,2),(1024*num_col,1024*num_row,3))
print("shape of reconstructed wsi is {}".format(stitched_wsi.shape))

Number of images processed: 100%|██████████| 13774/13774 [00:00<00:00, 1967789.89it/s]


time it took: 0.0 minutes
min_x is 1126, min_y is 1126, num_row is 145, num_col is 95


Number of images processed: 100%|██████████| 13774/13774 [10:54<00:00, 21.04it/s]


time it took: 10.916666666666666 minutes
shape of tiles is (95, 145, 1024, 1024, 3)
shape of reconstructed wsi is (97280, 148480, 3)


In [5]:
start = time.time()
im = pyvips.Image.new_from_array(stitched_wsi)
image_height = im.height
image_bands = im.bands
print("imheight imwidth",image_height, im.width)
im = im.copy()
im.set_type(pyvips.GValue.gint_type, "page-height", image_height)
im.set_type(pyvips.GValue.gstr_type, "image-description",
            f"""<?xml version="1.0" encoding="UTF-8"?>
<OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
    xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
    <Image ID="Image:0">
        <!-- Minimum required fields about image dimensions -->
        <Pixels DimensionOrder="XYCZT"
                ID="Pixels:0"
                SizeC="{image_bands}"
                SizeT="1"
                SizeX="{im.width}"
                SizeY="{image_height}"
                SizeZ="1"
                Type="uint8">
        </Pixels>
    </Image>
</OME>""")
savesrc = r'\\shelter\Kyu\unstain2stain\tiles\registered_stitched'
end = time.time()
print("time it took to set ometiff up took {} minutes".format((round(end-start)/60),3))

start = time.time()
im.tiffsave(os.path.join(savesrc, 'OTS_14832_10_he.ome.tiff'), compression="jpeg", tile=True, tile_width=512,
            tile_height=512, pyramid=True, subifd=True)
end = time.time()
print("time it took to save ometiff took {} minutes".format((round(end-start)/60),3))

imheight imwidth 97280 148480
time it took to set ometiff up took 0.0 minutes
time it took to save ometiff took 9.333333333333334 minutes


# STITCH WORKFLOW IN ONE FUNCTION:

In [1]:
import os
vipshome = r'C:\Users\Kevin\Documents\vips-dev-8.14\bin' #download libvips for windows, this is the bin path
os.environ['PATH'] = vipshome + ';' + os.environ['PATH']
import pyvips
import numpy as np
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import time
import shutil
from tqdm import tqdm

def stitch_tiles_to_ometiff(tile_dir,ometiff_save_dir, ometiff_name):
    imlist = [x for x in os.listdir(tile_dir) if x.endswith('.png')]
    min_x = float('inf')
    min_y = float('inf')
    max_x = float('-inf')
    max_y = float('-inf')
    start = time.time()
    print("counting number of rows and columns to stitch")
    for filename in tqdm(imlist, desc="Number of images processed", colour = 'red'):
        x, y = map(int, filename.split('xy')[0].split('.png')[0].split('_'))
        if x < min_x:
            min_x = x
        if y < min_y:
            min_y = y
        if x > max_x:
            max_x = x
        if y > max_y:
            max_y = y
    num_row = int((max_x + 1024 - min_x)/1024)
    num_col = int((max_y + 1024 - min_y)/1024)
    array_5d = np.zeros((num_col,num_row, 1024, 1024, 3), dtype=np.uint8)
    print("stitching images")
    for filename in tqdm(imlist, desc="Number of images processed", colour = 'red'):
        x, y = map(int, filename.split('xy')[0].split('.png')[0].split('_'))
        x = int((x-min_x)/1024)
        y = int((y-min_y)/1024)
        image = Image.open(os.path.join(tile_dir, filename))
        image = np.array(image)
        array_5d[y,x, :, :, :] = image
    stitched_wsi = np.reshape(array_5d.swapaxes(1,2),(1024*num_col,1024*num_row,3))
    print("shape of reconstructed wsi is {}".format(stitched_wsi.shape))
    end = time.time()
    print("time it took to create reconstructed wsi is: {} minutes".format((round(end-start)/60),3))
    print("--- saving as ometiff ---")
    start = time.time()
    im = pyvips.Image.new_from_array(stitched_wsi)
    image_height = im.height
    image_bands = im.bands
    im = im.copy()
    im.set_type(pyvips.GValue.gint_type, "page-height", image_height)
    im.set_type(pyvips.GValue.gstr_type, "image-description",
                f"""<?xml version="1.0" encoding="UTF-8"?>
    <OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
        xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
        xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
        <Image ID="Image:0">
            <!-- Minimum required fields about image dimensions -->
            <Pixels DimensionOrder="XYCZT"
                    ID="Pixels:0"
                    SizeC="{image_bands}"
                    SizeT="1"
                    SizeX="{im.width}"
                    SizeY="{image_height}"
                    SizeZ="1"
                    Type="uint8">
            </Pixels>
        </Image>
    </OME>""")

    im.tiffsave(os.path.join(ometiff_save_dir, ometiff_name), compression="jpeg", tile=True, tile_width=512,
                tile_height=512, pyramid=True, subifd=True)
    end = time.time()
    print("time it took to save ometiff took {} minutes".format((round(end-start)/60),3))
    print("ometiff saved successfully!")

In [4]:
stitch_tiles_to_ometiff(tile_dir = r'\\shelter\Kyu\unstain2stain\tiles\registered_tiles\US\OTS_14684_7', ometiff_save_dir = r'\\shelter\Kyu\unstain2stain\tiles\registered_stitched\US',ometiff_name = 'OTS_14684_7.ome.tiff')# os.path.join ometiff_save_dir, ometiff_name = full save path

counting number of rows and columns to stitch


Number of images processed: 100%|██████████| 10545/10545 [00:00<00:00, 1763232.96it/s]


stitching images


Number of images processed: 100%|██████████| 10545/10545 [09:48<00:00, 17.92it/s]


shape of reconstructed wsi is (97280, 113664, 3)
time it took to create reconstructed wsi is: 9.983333333333333 minutes
--- saving as ometiff ---
time it took to save ometiff took 7.166666666666667 minutes
ometiff saved successfully!


### Try registering the two tiles with optical flow to compare with matlab rigid registration:


In [11]:
def registrate_two_images(reference_image_path, moving_image_path, save_path):
    """
    Note: The order of the files saved in the ref_img_path and mov_img_path must be the same so that you are registering the same images!
    """
    ref_img_path = [_ for _ in os.listdir(reference_image_path) if _.endswith(".png")]
    ref_img_path_complete = [os.path.join(reference_image_path, x) for x in ref_img_path]
    mov_img_path = [_ for _ in os.listdir(moving_image_path) if _.endswith(".png")]
    mov_img_path_complete = [os.path.join(moving_image_path, x) for x in mov_img_path]
    mov_img_name = [x.replace('.png','') for x in mov_img_path]
    if int(len(ref_img_path)) != int(len(mov_img_path)):
        print("Number of images in reference and moving file paths are not equal, please fix and try again!")
        return

    start = time()
    for idx in range(0,len(ref_img_path_complete)):
        ref_img = Image.open(ref_img_path_complete[idx])
        mov_img = Image.open(mov_img_path_complete[idx])
        ref_img = np.array(ref_img)
        mov_img = np.array(mov_img)
        ref_img_g = cv2.cvtColor(ref_img,cv2.COLOR_RGBA2GRAY)
        mov_img_g = cv2.cvtColor(mov_img,cv2.COLOR_RGBA2GRAY)
        v, u = optical_flow_tvl1(ref_img_g, mov_img_g)
        nr, nc = ref_img_g.shape
        row_coords, col_coords = np.meshgrid(np.arange(nr), np.arange(nc),
                                             indexing='ij')
        mov_img_warp_ra =[]
        for i in range(3):
            mov_img_warp = warp(mov_img[:,:,i], np.array([row_coords + v, col_coords + u]),mode='edge')
            mov_img_warp_ra.append(mov_img_warp)
        r = np.array(mov_img_warp_ra[0]*255).astype('uint8')
        g = np.array(mov_img_warp_ra[1]*255).astype('uint8')
        b = np.array(mov_img_warp_ra[2]*255).astype('uint8')
        rgb = np.stack([r,g,b],axis=2)
        reg_img = Image.fromarray(rgb)
        print(idx)
        reg_img.save(os.path.join(save_path,str(mov_img_name[idx]) + '.png'))

    end = time()
    print("time it took to register: "+  str((end-start)/60) + " minutes")


### Possible dummy code to recognize H&E images and discard images that are not H&E:

In [32]:
import os
import cv2

# Define the directory where the image tiles are stored
img_dir = r'\\shelter\Kyu\unstain2stain\unstain2stain_tile\train\HE\OTS_14684_3_he - Copy'

# Define the threshold value for identifying empty tiles
threshold_value = 200
removed_name = []
# Loop through all the image files in the directory
for filename in os.listdir(img_dir):
    # Load the image
    img = cv2.imread(os.path.join(img_dir, filename))

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Calculate the mean pixel value of the grayscale image
    mean_pixel_value = cv2.mean(gray)[0]
    #maybe put median?

    # If the mean pixel value is below the threshold, assume the tile is empty and discard it
    if mean_pixel_value < threshold_value:
        os.remove(os.path.join(img_dir, filename))
        print('Removed:', filename)
        removed_name.append(filename)
    else:
        print('Kept:', filename)


C:\Users\Kevin\AppData\Local\Temp\ipykernel_24640\2577889262.py:22: RuntimeWarning: invalid value encountered in divide
  bgr = (blue - (red + green) / 2) / (blue + green + red)


54


In [ ]:
### Code to drop out low MI tiles: